# Getting model "cast" files into one dataframe with depth matched to observations

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import gsw
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [2]:
def nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [5]:
path = Path("/data1/bbeutel/LO_output/extract_cast/")

In [24]:
def read_dfo_ctd(year):    

    dfo = pd.read_pickle('/data1/parker/LO_output/obs/dfo1/ctd/'+str(year)+'.p')

    dfo['h'] = np.nan
    dfo['z_model'] = np.nan
    dfo['PT_model'] = np.nan
    dfo['SA_model'] = np.nan
    dfo['CT_model'] = np.nan
    dfo['DO_model'] = np.nan
    cid = np.arange(0,int(np.max(dfo.cid))+1,1)

    files = [sorted(path.glob("dfo1_ctd_{}/{}.nc".format(year,str(cid[i])))) for i in range(len(cid))]

    for i in range(len(files)):
        if len(files[i]) == 1:
            # print(i)
            cast = xr.open_dataset(files[i][0])

            dfo.h[dfo.cid == i] = 
            dfo.z_model[dfo.cid == i] = np.array([nearest(cast.h*cast.s_rho,dfo.z[dfo.cid == i][j]) 
                                                        for j in np.array(dfo[dfo.cid == i].index)])
            dfo.PT_model[dfo.cid == i] = np.array([cast.temp[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            dfo.SA_model[dfo.cid == i] = np.array([cast.salt[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            dfo.DO_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])


    dfo.CT_model = gsw.CT_from_pt(dfo.SA_model, dfo.PT_model)

    name = "dfo_ctd_"+str(year)+".csv"
    dfo.to_csv(name)

In [27]:
read_dfo_ctd(2018)

In [28]:
read_dfo_ctd(2021)

KeyboardInterrupt: 

In [22]:
def read_dfo_bottle(year):    

    dfo = pd.read_pickle('/data1/parker/LO_output/obs/dfo1/bottle/'+str(year)+'.p')

    dfo['z_model'] = np.nan
    dfo['PT_model'] = np.nan
    dfo['SA_model'] = np.nan
    dfo['CT_model'] = np.nan
    dfo['DO_model'] = np.nan
    dfo['NO3_model'] = np.nan
    dfo['Chl_model'] = np.nan

    cid = np.arange(0,int(np.max(dfo.cid))+1,1)

    files = [sorted(path.glob("dfo1_ctd_2019/{}.nc".format(str(cid[i])))) for i in range(len(cid))]

    for i in range(len(files)):
        if len(files[i]) == 1:
            # print(i)
            cast = xr.open_dataset(files[i][0])
            dfo.z_model[dfo.cid == i] = np.array([nearest(cast.h*cast.s_rho,dfo.z[dfo.cid == i][j]) 
                                                        for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            dfo.PT_model[dfo.cid == i] = np.array([cast.temp[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            dfo.SA_model[dfo.cid == i] = np.array([cast.salt[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            dfo.DO_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            dfo.NO3_model[dfo.cid == i] = np.array([cast.NO3[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            dfo.Chl_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index) if type(dfo['NO3 (uM)'][j])==float])
            # dont really know what to do with Chl yet but might as well take it


    dfo.CT_model = gsw.CT_from_pt(dfo.SA_model, dfo.PT_model)

    name = "dfo_botte_"+str(year)+".csv"
    dfo.to_csv(name)

In [23]:
read_dfo_bottle(2017)

type: cannot set using a list-like indexer with a different length than the value